# Summary Evaluators

### Setup

In [32]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

### Task

Our task here is to analyze the toxictity of random statements, classifying them as `Toxic` or `Not toxic`. 

Take a look at our dataset!

In [33]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
    "https://smith.langchain.com/public/35d51105-a2fd-46cb-8945-5022b3a09dae/d"
)

This is a simple toxicity classifier!

In [34]:
from openai import OpenAI

from pydantic import BaseModel, Field

openai_client = OpenAI()

class Toxicity(BaseModel):
    toxicity: str = Field(description="""'Sarcasm' if this the statement is Sarcastic, 'Not Sarcastic' if the statement is not sarcastic.""")

def good_classifier(inputs: dict) -> dict:
    completion = openai_client.beta.chat.completions.parse(
        model="gpt-5-nano",
        messages=[
            {
                "role": "user",
                "content": f"This is the statement: {inputs['Input']}"
            }
        ],
        response_format=Toxicity,
    )

    toxicity_score = completion.choices[0].message.parsed.toxicity
    return {"Sarcasm": toxicity_score}

### Summary Evaluator

These are the fields that summary evaluator functions get access to:
- `inputs: list[dict]`: A list of inputs from the examples in our dataset
- `outputs: list[dict]`: A list of the dict outputs produced from running our target over each input
- `reference_outputs: list[dict]`: A list of reference_outputs from the examples in our dataset
- `runs: list[Run]`: A list of the Run objects from running our target over the dataset.
- `examples: list[Example]`: A list of the full dataset Examples, including the example inputs, outputs (if available), and metdata (if available).

Now we'll define our summary evaluator! Here, we'll compute the f1-score, which is a combination of precision and recall.

This sort of metric can only be computed over all of the examples in our experiment, so our evaluator takes in a list of outputs, and a list of reference_outputs.

In [35]:
def f1_score_summary_evaluator(outputs: list[dict], reference_outputs: list[dict]) -> dict:
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for output_dict, reference_output_dict in zip(outputs, reference_outputs):
        output = output_dict["Sarcasm"]
        reference_output = reference_output_dict["Sarcasm"]
        if output == "Sarcasm" and reference_output == "Sarcasm":
            true_positives += 1
        elif output == "Sarcasm" and reference_output == "Not Sarcastic":
            false_positives += 1
        elif output == "Not Sarcastic" and reference_output == "Sarcasm":
            false_negatives += 1

    if true_positives == 0:
        return {"key": "f1_score", "score": 0.0}

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return {"key": "f1_score", "score": f1_score}


Note that we pass in `f1_score_summary_evaluator` as a summary evaluator!

In [36]:
results = client.evaluate(
    good_classifier,
    data=dataset,
    summary_evaluators=[f1_score_summary_evaluator],
    experiment_prefix="Good classifier"
)

View the evaluation results for experiment: 'Good classifier-8141ad66' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/cfedccbe-c89c-41a2-a71b-9f559183586a/compare?selectedSessions=a06027fa-f70f-442c-b57f-9c2400f5ff79




0it [00:00, ?it/s]